# Data extraction

The following code, extracts, loads, subsets and plots data used in the analysis and to prepare figures for the paper. These include:

 -  data on global production of buckwhat, millet and rice available from [EarthStat](http://www.earthstat.org/)
 -  outline of the continents and the shaded relief map combined with custom elevation colors from [Natural Earth](https://www.naturalearthdata.com/)
 -  administrative division of China available from [DIVA-GIS](https://www.diva-gis.org/gdata)
 -  data on climatic conditions for the past 120 000 years made available by [Beyer et al. 2019](https://doi.org/10.1038/S41597-020-0552-1) ([figshare repository](https://figshare.com/articles/dataset/LateQuaternary_Environment_nc/12293345))
 -  pollen recrods available from ([Cao et al. 2013](https://doi.org/10.1016/j.revpalbo.2013.02.003)), and obtained from the authors (original file not provided here only the subset with *Fagopyrum* records)
 -  dataset with the locations of fagopyrum macro and micro remains from China, based on [Hunt et al. 2018](https://doi.org/10.1007/s00334-017-0649-4), and available in the project's repository (original data sources for individual sites are provided in [ESM 3](../data/ESM_3.pdf)).

In [ ]:
### Load libraries:
library(here) # setting paths
library(raster) # for spatial data
library(rgdal) # for spatial data
library("rioja") # pollen plots
library(maptools) # making maps with plotRGB
library(prettymapr) # for north arrow
library(tmap) # map visualisations
library(stringr) # wraps labels on the maps
library(ncdf4) # loading climate data

In [2]:
crops<-c("millet","rice","wheat","barley","buckwheat")

In [3]:
crop_fnames<-paste(here("raw_data"),crops,"_HarvAreaYield_Geotiff",sep="")


In [4]:
crop_fnames<-paste(here("raw_data"),"/",crops,"_HarvAreaYield_Geotiff",sep="")


In [5]:
### Define input files:

path2timeslices<-here("outputs","chronology_limits.csv")

In [6]:
### Define urls from which to download data and paths to their location in the repository:

# Production data:
buckwheat_url <- "https://s3.us-east-2.amazonaws.com/earthstatdata/HarvestedAreaYield175Crops_Indvidual_Geotiff/buckwheat_HarvAreaYield_Geotiff.zip"
# Also define the name of the folder with the production data, taken from the path
buckwheat_fname<-here("raw_data","buckwheat_HarvAreaYield_Geotiff")

crop_urls<-paste("https://s3.us-east-2.amazonaws.com/earthstatdata/HarvestedAreaYield175Crops_Indvidual_Geotiff/",crops,"_HarvAreaYield_Geotiff.zip",sep="")
crop_fnames<-paste(here("raw_data"),"/",crops,"_HarvAreaYield_Geotiff",sep="")

#millet_url<-"https://s3.us-east-2.amazonaws.com/earthstatdata/HarvestedAreaYield175Crops_Indvidual_Geotiff/millet_HarvAreaYield_Geotiff.zip"
#millet_fname<-here("raw_data","millet_HarvAreaYield_Geotiff")

#rice_url<-"https://s3.us-east-2.amazonaws.com/earthstatdata/HarvestedAreaYield175Crops_Indvidual_Geotiff/rice_HarvAreaYield_Geotiff.zip"
#rice_fname<-here("raw_data","rice_HarvAreaYield_Geotiff")

#wheat_url<-"https://s3.us-east-2.amazonaws.com/earthstatdata/HarvestedAreaYield175Crops_Indvidual_Geotiff/wheat_HarvAreaYield_Geotiff.zip"
#wheat_fname<-here("raw_data","wheat_HarvAreaYield_Geotiff")

#barley_url<-"https://s3.us-east-2.amazonaws.com/earthstatdata/HarvestedAreaYield175Crops_Indvidual_Geotiff/barley_HarvAreaYield_Geotiff.zip"
#barley_fname<-here("raw_data","barley_HarvAreaYield_Geotiff")


# Outline of the continents:
cont_url<-"https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/physical/ne_50m_land.zip"
# Also define the name of the folder with continents, taken from the path
cont_fname<-here("raw_data","ne_50m_land")

# Administrative division of China:
china_url<-"http://biogeo.ucdavis.edu/data/diva/adm/CHN_adm.zip"
# Define the directory to which the files will be extracted
china_fname <- here("raw_data","CHN_adm")

# Environmental variables:
env_url <- "https://ndownloader.figshare.com/articles/12293345/versions/3"
# Define the directory to which the files will be extracted
env_fname <- here("raw_data","12293345")

# Shaded relief map of the world:

ne_url <- "http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/raster/HYP_HR_SR_W_DR.zip"
ne_fname <-here("raw_data","HYP_HR_SR_W_DR")

# Pollen data from Cao et al 2013:
path2pollen<-here("raw_data","FossilPollenDatabasePercentageAsian.csv")

# Fagopyrum locations:

path2locations<-here("data","ESM_2.csv")

# Path to raw data folder
path2raw_data<- here("raw_data")

In [7]:
### Define the output paths:

# Subsets of production data
path2production<-here("data","production")
path2clipped_china<-here("data","production","clipped")
path2masked_china<-here("data","production","masked")

# Plots of production in the world and in China
path2world_production <- here("outputs","ESM_4.tiff")
path2world_millet_production <- here("outputs","gprod_millet.tiff")
path2world_rice_production <- here("outputs","gprod_rice.tiff")

gprod_paths<-paste(here("outputs"),"/","global_production_",crops,".tiff",sep="")

path2china_production <- here("outputs","ESM_5.tiff")

# Table with full names and short names of environmental variables:

path2env_table<-here("outputs","Predictor_variables.csv")

# Past environmental layers:

path2past_env<-here("data","environmental","past")
path2present_env<-here("data","environmental","present")

# Pollen diagrams

path2pollen_diag<-here("outputs","ESM_1.pdf")

# Map of site locations (figure 1)

path2locations_map<-here("outputs","Fig1.tiff")

In [ ]:
# Create directory to downlaod raw data into:
dir.create(path2raw_data)
dir.create(path2present_env)

In [8]:
### Load utility functions that will be used in data processing and plots:
source(here("R","extractRastersFromNetCDF.r"))
source(here("R","theme_tmaps.R"))

## Maps backgrounds for plotting

In [5]:
### Download the shapefile of the continents as well:

# Download the zipped folder 
download.file(cont_url,paste(cont_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
files <- unzip(paste(cont_fname,".zip",sep=""),exdir=cont_fname)
# Remove the zipped folder
file.remove(paste(cont_fname,".zip",sep=""))

[1] TRUE

In [36]:
# Load the shapefile with the outline of the continents
continents <- readOGR(dsn = cont_fname, layer = sub('.*/', '', cont_fname))

OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\my_repositories\PhD\raw_data\ne_50m_land", layer: "ne_50m_land"
with 1420 features
It has 3 fields
Integer64 fields read as strings:  scalerank 


## Present distribution of crops

### Extract global production

Data on the present global production of buckwhat is available from [EarthStat](http://www.earthstat.org/) and contains information on the volume of production, cultivated area and yield, as well as data quality. In the model it will be used as a proxy for the suitabiliy of environment for buckwheat cultivation (response variable). The following code downloads the data from EarthStat and extracts the individual files from the archive. Then it also downloads and loads the files with the outline of the continents from [NaturalEarth](https://www.naturalearthdata.com/downloads/50m-physical-vectors/) to serve as a background for data visualisation and plots the visualisation.

In [ ]:
### Download and extract the production data

# Buckwheat
#download.file(prod_url,paste(prod_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
#prod_files <- unzip(paste(prod_fname,".zip",sep=""),exdir=path2raw_data)
# Remove the zipped folder
#file.remove(paste(prod_fname,".zip",sep=""))

In [7]:
# Millet

download.file(millet_url,paste(millet_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
millet_files <- unzip(paste(millet_fname,".zip",sep=""),exdir=path2raw_data)
# Remove the zipped folder
file.remove(paste(millet_fname,".zip",sep=""))


[1] TRUE

In [9]:
download.file(rice_url,paste(rice_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
rice_files <- unzip(paste(rice_fname,".zip",sep=""),exdir=path2raw_data)
# Remove the zipped folder
file.remove(paste(rice_fname,".zip",sep=""))

[1] TRUE

In [6]:
download.file(wheat_url,paste(wheat_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
wheat_files <- unzip(paste(wheat_fname,".zip",sep=""),exdir=path2raw_data)
# Remove the zipped folder
file.remove(paste(wheat_fname,".zip",sep=""))

[1] TRUE

In [7]:
download.file(barley_url,paste(barley_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
barley_files <- unzip(paste(barley_fname,".zip",sep=""),exdir=path2raw_data)
# Remove the zipped folder
file.remove(paste(barley_fname,".zip",sep=""))

[1] TRUE

In [6]:
### Load data into workspace:

# Millet
# List all layers related to buckwheat production
millet_layers<-list.files(path=millet_fname,pattern='tif$',full.names=TRUE)
# Put all the layers into a raster stack
millet<-stack(millet_layers)

In [7]:
# Rice
# List all layers related to buckwheat production
rice_layers<-list.files(path=rice_fname,pattern='tif$',full.names=TRUE)
# Put all the layers into a raster stack
rice<-stack(rice_layers)

In [69]:
layers<-lapply(crop_fnames,list.files,pattern='tif$',full.names=TRUE)

In [72]:
crop_data<-lapply(layers,stack)
correct_layer<-crop_data[[1]][[3]]
crop_data<-lapply(crop_data,function(x,correct_layer){ x[is.na(correct_layer)]<-NA 
                  return(x)},correct_layer)

In [34]:
# Look at the existing names of the layers in the raster stack
names(crop_data[[1]])
# Create a list of more informative names
prod_names <- c("Data quality: harvested area","Data quality: yield","Harvested area: fractional","Harvested area: hectares",
            "Production in tons","Yield (tons per hectare)")
# Prepare the key from the census level of data quality layers (available from metadata pdf in a downloaded folder) 
census_level <-c("missing census data","country level census data","interpolated from within 2° lat/lon","state level census data","county level census data")

#Make a color palette for visualisation
pal <- colorRampPalette(c("lightyellow","orange","orange3","saddlebrown"))

# Set NA's to the cells that represent the seas in the data quality layers:
#millet[[1]][is.na(millet[[3]])]<-NA
#millet[[2]][is.na(millet[[3]])]<-NA

#rice[is.na(millet[[3]])]<-NA

[1] "millet_DataQuality_HarvestedArea" "millet_DataQuality_Yield"        
[3] "millet_HarvestedAreaFraction"     "millet_HarvestedAreaHectares"    
[5] "millet_Production"                "millet_YieldPerHectare"

In [76]:
options(repr.plot.width=7,repr.plot.height=7,repr.plot.res=250)
gmaps<-c()
### Plot global production data
for (i in 1:length(crops)){
    print(i)
# Make panels for the data quality:
gprod1<-tm_shape(crop_data[[i]][[c(1,2)]]) + # Order the maps in a sensible way
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=2,labels.size = 0.6)+ #Add the lat long lines
    # Add raster layers:
    tm_raster(labels=str_wrap(census_level,40),style=c("cat","cat"),palette=pal(4),legend.reverse = TRUE,title="",interval.closure="left",showNA=FALSE,colorNA="NA",midpoint=NULL)+
    # Make facets
    tm_facets(free.scales=TRUE,free.coords=TRUE,ncol=2,nrow=1,free.scales.raster=TRUE)+
    # Add continents outline
    tm_shape(continents)+
    tm_borders(col="grey85")+
    # Change theme
    theme_tmaps+
    # Change layout
    tm_layout(panel.labels=prod_names[1:2], 
              panel.label.height=2,
              panel.label.size=0.8,
              legend.text.size=0.45,
              legend.position=c(0,0.075),
             )

# Make panels for the production data:
gprod2<-tm_shape(crop_data[[i]][[c(3,5,4,6)]]) + # Order the maps in a sensible way
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=2,labels.size = 0.8)+ #Add the lat long lines
    # Add raster layer:
    tm_raster(style=c("cont","cont","cont","cont"),palette=pal(4),legend.reverse = TRUE,title="",interval.closure="left",showNA=FALSE,colorNA="NA",midpoint=NULL)+
    # Make facets
    tm_facets(free.scales=TRUE,free.coords=TRUE,ncol=2,nrow=2,free.scales.raster=TRUE)+
    # Add continents outline
    tm_shape(continents)+
    tm_borders(col="grey85")+
    theme_tmaps+
    tm_layout(panel.labels=prod_names[c(3,5,4,6)],
              legend.position=c(0,0),
              panel.label.height=2,
              panel.label.size=1,
              legend.text.size=0.6
             )+
# Add info about the data projection
tm_credits("Coordinate system: World Geodetic System 1984\nData source: http://www.earthstat.org",position=c("left","bottom"),)

# Arrenge the maps together
gprod<-tmap_arrange(gprod1, gprod2, heights=c(1/3,2/3))

#Optional: plot the maps
gmaps[[i]]<-gprod
}

[1] 1
[1] 2
[1] 3
[1] 4


In [ ]:
# Save the plot as ESM 4
width=17.4
height=17.4
for(i in 1:length(crops)){
tmap_save(tm = gmaps[[i]], filename = gprod_paths[i], width = width, height = height,units = "cm",dpi = 600)}

### Subset production for China

The following code also downloads the shapefile with the administrative division of China from [DIVA-GIS](http://biogeo.ucdavis.edu/data/diva/adm/CHN_adm.zip), which is then used to crop, mask and plot data on buckwheat production in China. Is saves the cropped and masked subests of data as GeoTiffs for convenient access.

In [9]:
paste(path2raw_data,"\\","china_fname")

[1] "G:/My Drive/my_repositories/PhD/raw_data \\ china_fname"

In [10]:
paste(path2raw_data,"\\",china_fname,sep="")

[1] "G:/My Drive/my_repositories/PhD/raw_data\\G:/My Drive/my_repositories/PhD/raw_data/CHN_adm"

In [90]:
### Download and laod data on the administrative division of Chin
# Download the zipped folder
#download.file(china_url,paste(china_fname,".zip",sep=""), mode = "wb")
dir.create(china_fname)
# Unzip the folder and get the names of its contents:
files <- unzip(paste(china_fname,".zip",sep=""),exdir=china_fname)
# Removes the zipped folder:
file.remove(paste(china_fname,".zip",sep=""))

[1] TRUE

In [11]:
### Load the shapefile with the borders of China
china <- readOGR(dsn = china_fname, layer = "CHN_adm0")

OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\my_repositories\PhD\raw_data\CHN_adm", layer: "CHN_adm0"
with 1 features
It has 70 fields
Integer64 fields read as strings:  ID_0 OBJECTID_1 


In [19]:
### Create directories for subset data
dir.create(path2clipped_china)
dir.create(path2masked_china)

In [80]:
### Subset data for China:


for(i in 3:4){
path2clipped<-paste(path2clipped_china,"//",crops[i],sep="")
dir.create(path2clipped)
path2masked<-paste(path2masked_china,"//",crops[i],sep="")
dir.create(path2masked)
# Crop the data to the extent of china
clipped_prd<-crop(crop_data[[i]], extent(china), snap="out") # Note that china has the same projection as the prd maps, so it does not need to be reprojected
# Save clipped data as GeoTiffs:
writeRaster(clipped_prd,filename=paste(path2clipped,"//",names(crop_data[[i]]) ,".tif",sep=""),format="GTiff", overwrite=TRUE,bylayer=TRUE)
# Mask the data to the borders of China
masked_prd<-mask(clipped_prd, china)
# Saved masked data as GeoTiffs
writeRaster(masked_prd,filename=paste(path2masked,"//",names(crop_data[[i]]),".tif",sep=""),format="GTiff", overwrite=TRUE,bylayer=TRUE)}

In [42]:
masked_fnames<-paste(here("data","production","masked"),"/",crops,sep="")
layers<-lapply(masked_fnames,list.files,pattern='tif$',full.names=TRUE)

In [43]:
crop_data<-lapply(layers,stack)
correct_layer<-crop_data[[1]][[3]]
crop_data<-lapply(crop_data,function(x,correct_layer){ x[is.na(correct_layer)]<-NA 
                  return(x)},correct_layer)

In [44]:
# Look at the existing names of the layers in the raster stack
names(crop_data[[1]])
# Create a list of more informative names
prod_names <- c("Data quality: harvested area","Data quality: yield","Harvested area: fractional","Harvested area: hectares",
            "Production in tons","Yield (tons per hectare)")
# Prepare the key from the census level of data quality layers (available from metadata pdf in a downloaded folder) 
census_level <-c("missing census data","country level census data","interpolated from within 2° lat/lon","state level census data","county level census data")

#Make a color palette for visualisation
pal <- colorRampPalette(c("lightyellow","orange","orange3","saddlebrown"))

# Set NA's to the cells that represent the seas in the data quality layers:
#millet[[1]][is.na(millet[[3]])]<-NA
#millet[[2]][is.na(millet[[3]])]<-NA

#rice[is.na(millet[[3]])]<-NA

[1] "millet_DataQuality_HarvestedArea" "millet_DataQuality_Yield"        
[3] "millet_HarvestedAreaFraction"     "millet_HarvestedAreaHectares"    
[5] "millet_Production"                "millet_YieldPerHectare"

In [45]:
options(repr.plot.width=7,repr.plot.height=7,repr.plot.res=250)
gmaps<-c()
### Plot global production data
for (i in 1:length(crops)){
    print(i)
# Make panels for the data quality:
gprod1<-tm_shape(crop_data[[i]][[c(1,2)]]) + # Order the maps in a sensible way
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=2,labels.size = 0.6)+ #Add the lat long lines
    # Add raster layers:
    tm_raster(labels=str_wrap(census_level,40),style=c("cat","cat"),palette=pal(4),legend.reverse = TRUE,title="",interval.closure="left",showNA=FALSE,colorNA="NA",midpoint=NULL)+
    # Make facets
    tm_facets(free.scales=TRUE,free.coords=TRUE,ncol=2,nrow=1,free.scales.raster=TRUE)+
    # Add continents outline
    tm_shape(continents)+
    tm_borders(col="grey85")+
    # Change theme
    theme_tmaps+
    # Change layout
    tm_layout(panel.labels=prod_names[1:2], 
              panel.label.height=2,
              panel.label.size=0.8,
              legend.text.size=0.45,
              legend.position=c(0,0.075),
             )

# Make panels for the production data:
gprod2<-tm_shape(crop_data[[i]][[c(3,5,4,6)]]) + # Order the maps in a sensible way
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=2,labels.size = 0.8)+ #Add the lat long lines
    # Add raster layer:
    tm_raster(style=c("cont","cont","cont","cont"),palette=pal(4),legend.reverse = TRUE,title="",interval.closure="left",showNA=FALSE,colorNA="NA",midpoint=NULL)+
    # Make facets
    tm_facets(free.scales=TRUE,free.coords=TRUE,ncol=2,nrow=2,free.scales.raster=TRUE)+
    # Add continents outline
    tm_shape(continents)+
    tm_borders(col="grey85")+
    theme_tmaps+
    tm_layout(panel.labels=prod_names[c(3,5,4,6)],
              legend.position=c(0,0),
              panel.label.height=2,
              panel.label.size=1,
              legend.text.size=0.6
             )+
# Add info about the data projection
tm_credits("Coordinate system: World Geodetic System 1984\nData source: http://www.earthstat.org",position=c("left","bottom"),)

# Arrenge the maps together
gprod<-tmap_arrange(gprod1, gprod2, heights=c(1/3,2/3))

#Optional: plot the maps
gmaps[[i]]<-gprod
}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5


In [ ]:
# Save the plot as ESM 4
width=17.4
height=17.4
for(i in 1:length(crops)){
tmap_save(tm = gmaps[[i]], filename = gprod_paths_local[i], width = width, height = height,units = "cm",dpi = 300)}

### Environmental data

In [4]:
### Preparing environmental data for 6 time slices from Mid-Holocene to the present:

# Download the zipped folder
download.file(env_url,paste(env_fname,".zip",sep=""), mode = "wb")

# Unzip the folder and get the names of its contents:
files <- unzip(paste(env_fname,".zip",sep=""), exdir=env_fname)

# Removes the zipped folder:
file.remove(paste(env_fname,".zip",sep=""))

[1] TRUE

In [12]:
files<-c(here("raw_data","12293345","LateQuaternary_Environment.nc"))

In [25]:
### Extract the relevant layers from NetCDF and saves them as raster:

# Open the NetCDF, in order to get the names of the variables that are stored there and summarizes the time dimension of the data
env_nc <- ncdf4::nc_open(files[1])
# Extracts the availabe variable names:
var_names <- names(env_nc$var)
#Included only those relevant for the analysis
short_names <-var_names[c(9,11:27)] 
# Define relevant time slices (past 8000 years + 15000 BP)
timeslices<-read.csv(path2timeslices,row.names=1)
timeslices<-as.numeric(gsub(".BP","",gsub("X","",colnames(timeslices))))
# Define the area to mask and crop the rasters by
area<-china

In [28]:
source(here("R","extractRastersFromNetCDF.r"))


In [29]:
# Extract the relevant layers from the NetCDF file, using the utility function provided in this repository
bricks<-lapply(short_names,extractRastersFromNetCDF,path2file=files[1],timeslice=timeslices[11:15],area=area,path2layers=path2past_env,
       path2present=path2present_env,crop=TRUE,mask=TRUE )

Warning message in dir.create(paste(path2present, "\\cropped\\", sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\present\cropped' already exists"
Warning message in dir.create(paste(path2present, "\\masked\\", sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\present\masked' already exists"
Warning message in dir.create(path2layers):
"'G:\My Drive\my_repositories\PhD\data\environmental\past' already exists"
Warning message in dir.create(paste(path2layers, "\\cropped\\", variable, sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\past\cropped\npp' already exists"
Warning message in dir.create(paste(path2layers, "\\masked\\", variable, sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\past\masked\npp' already exists"
Warning message in dir.create(paste(path2present, "\\cropped\\", sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\present\cropped' already exists"
Warning message in dir.create(paste(path2present, 

Warning message in dir.create(path2layers):
"'G:\My Drive\my_repositories\PhD\data\environmental\past' already exists"
Warning message in dir.create(paste(path2layers, "\\cropped\\", variable, sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\past\cropped\BIO12' already exists"
Warning message in dir.create(paste(path2layers, "\\masked\\", variable, sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\past\masked\BIO12' already exists"
Warning message in dir.create(paste(path2present, "\\cropped\\", sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\present\cropped' already exists"
Warning message in dir.create(paste(path2present, "\\masked\\", sep = "")):
"'G:\My Drive\my_repositories\PhD\data\environmental\present\masked' already exists"
Warning message in dir.create(path2layers):
"'G:\My Drive\my_repositories\PhD\data\environmental\past' already exists"
Warning message in dir.create(paste(path2layers, "\\cropped\\", variable, sep = "")):
"'G:

In [67]:
variable<-short_names[1]
timeslice=timeslices
area=area
path2file=files[1]
path2layers=path2past_env
path2present=path2present_env

In [36]:
r <- brick(path2file,varname = variable)


In [71]:
### Additionally, prepare the table with the shortcuts of environmental variables and their full names
full_name <-c("Net Primary Production",'BIO1 Annual Mean Temperature',
              'BIO4 Temperature Seasonality', 
              'BIO5 Max Temperature of Warmest Month',
              'BIO6 Min Temperature of Coldest Month', 
              'BIO7 Temperature Annual Range', 
              'BIO8 Mean Temperature of Wettest Quarter',
              'BIO9 Mean Temperature of Driest Quarter',
              'BIO10 Mean Temperature of Warmest Quarter', 
              'BIO11 Mean Temperature of Coldest Quarter', 
              'BIO12 Annual Precipitation', 
              'BIO13 Precipitation of Wettest Month', 
              'BIO14 Precipitation of Driest Month',
              'BIO15 Precipitation Seasonality',
              'BIO16 Precipitation of Wettest Quarter',
              'BIO17 Precipitation of Driest Quarter', 
              'BIO18 Precipitation of Warmest Quarter',
              'BIO19 Precipitation of Coldest Quarter')
env_table <- cbind(short_names, full_name)

# Write the names of environmental variables to a csv file for future reference
write.csv(env_table, file=path2env_table,row.names=FALSE)